# Language modelling


Обучим две различные символьные модели для генерации динозавров:
* модель на символьных биграмах
* ***RNN***-модель.


## Bigram model


In [0]:
!wget https://raw.githubusercontent.com/artemovae/NLP-seminar-LM/master/dinos.txt

--2019-09-23 17:51:32--  https://raw.githubusercontent.com/artemovae/NLP-seminar-LM/master/dinos.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.244.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.244.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19909 (19K) [text/plain]
Saving to: ‘dinos.txt’

dinos.txt           100%[===================>]  19,44K  --.-KB/s    in 0,02s   

2019-09-23 17:51:33 (827 KB/s) - ‘dinos.txt’ saved [19909/19909]



In [0]:
names = ['<' + name.strip().lower() + '>' for name in open('dinos.txt').readlines()]
print(names[:10])

['<aachenosaurus>', '<aardonyx>', '<abdallahsaurus>', '<abelisaurus>', '<abrictosaurus>', '<abrosaurus>', '<abydosaurus>', '<acanthopholis>', '<achelousaurus>', '<acheroraptor>']


In [0]:
#!pip install nltk

In [0]:
import nltk

Вычислим частоту каждого символа в корпусе имен динозавров

In [0]:
chars = [char  for name in names for char in name]
freq = nltk.FreqDist(chars)

In [0]:
print(list(freq.keys()))

['h', 'p', 'i', 'u', 'q', '<', 'b', 'a', 'r', 'x', 'g', 'e', 'o', 'l', 'c', 'j', 'z', 's', 'd', 'f', 'y', 'm', 'v', 'n', 'w', '>', 'k', 't']


In [0]:
freq.most_common(10)

[('a', 2487),
 ('s', 2285),
 ('u', 2123),
 ('o', 1710),
 ('r', 1704),
 ('<', 1536),
 ('>', 1536),
 ('n', 1081),
 ('i', 944),
 ('e', 913)]

Define a function to estimate probabilty of character

In [0]:
l = sum([freq[char] for char in freq])
def unigram_prob(char):
    return freq[char] / l

In [0]:
print('p(a) = %1.4f' %unigram_prob('a'))

p(a) = 0.1160


Вычислим условную вероятность каждого символа в зависимости от того, какой символ стоял на предыдущей позиции.

In [0]:
cfreq = nltk.ConditionalFreqDist(nltk.bigrams(chars))

In [0]:
cfreq['a']

FreqDist({'u': 791, 'n': 347, 't': 204, 's': 171, 'l': 138, '>': 138, 'r': 124, 'c': 100, 'p': 89, 'm': 68, ...})

Оценим условные вероятности с помощью MLE.

In [0]:
cprob = nltk.ConditionalProbDist(cfreq, nltk.MLEProbDist)

In [0]:
print('p(a a) = %1.4f' %cprob['a'].prob('a'))
print('p(a b) = %1.4f' %cprob['a'].prob('b'))
print('p(a u) = %1.4f' %cprob['a'].prob('u'))

p(a a) = 0.0044
p(a b) = 0.0097
p(a u) = 0.3181


In [0]:
cprob['a'].generate()

'n'

### Task 1.
a. Write a function to generate a dinosaur name of **fixed** length. Use '<' as a start of name symbol.

b. Write a to generate a dinosaur names of any length. 

## Реккурентные нейронные сети (RNN)

Исходная последовательность:

$x_{1:n} = x_1, x_2, \ldots, x_n$, $x_i \in \mathbb{R}^{d_{in}}$

Для каждого входного значения $x_{1:i}$ получаем на выходе $y_i$:

$y_i = RNN(x_{1:i})$, $y_i \in \mathbb{R}^{d_{out}}$

Для всей последовательности $x_{1:n}$:

$y_{1:n} = RNN^{*}(x_{1:n})$, $y_i \in \mathbb{R}^{d_{out}}$

$R$ - рекурсивная функция активации, зависящая от двух параметров: $x_i$ и $s_{i-1}$ (вектор предыдущего состояния)

$RNN^{*}(x_{1:n}, s_0) = y_{1:n}$

$y_i = O(s_i) = g(W^{out}[s_{i} ,x_i] +b)$

$s_i = R(s_{i-1}, x_i)$

$s_i = R(s_{i-1}, x_i) = g(W^{hid}[s_{i-1} ,x_i] +b)$  -- конкатенация $[s_{i-1}, x]$

$x_i \in \mathbb{R}^{d_{in}}$, $y_i \in \mathbb{R}^{ d_{out}}$, $s_i \in \mathbb{R}^{d_{hid}}$

$W^{hid} \in \mathbb{R}^{(d_{in}+d_{out}) \times d_{hid}}$, $W^{out} \in \mathbb{R}^{d_{hid} \times d_{out}}$

Построим языковую модель на основе RNN с помощью pytorch

In [0]:
import numpy as np
import random
import torch
import torch.nn as nn
import torch.optim as optim
import pdb
from torch.utils.data import Dataset, DataLoader

%load_ext autoreload
%autoreload 2

torch.set_printoptions(linewidth=200)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
hidden_size = 50

Подготовим данные

In [0]:
class DinosDataset(Dataset):
    def __init__(self):
        super().__init__()
        with open('dinos.txt') as f:
            content = f.read().lower()
            self.vocab = sorted(set(content)) + ['<', '>']
            self.vocab_size = len(self.vocab)
            self.lines = content.splitlines()
        self.ch_to_idx = {c:i for i, c in enumerate(self.vocab)}
        self.idx_to_ch = {i:c for i, c in enumerate(self.vocab)}
    
    def __getitem__(self, index):
        line = self.lines[index]
        #teacher forcing
        x_str = '<' + line 
        y_str = line + '>' 
        x = torch.zeros([len(x_str), self.vocab_size], dtype=torch.float)
        y = torch.empty(len(x_str), dtype=torch.long)
        for i, (x_ch, y_ch) in enumerate(zip(x_str, y_str)):
            x[i][self.ch_to_idx[x_ch]] = 1
            y[i] = self.ch_to_idx[y_ch]
        
        return x, y
    
    def __len__(self):
        return len(self.lines)

In [0]:
trn_ds = DinosDataset()
trn_dl = DataLoader(trn_ds, shuffle=True)

In [0]:
trn_ds.lines[1]

'aardonyx'

In [0]:
print(trn_ds.idx_to_ch)

{0: '\n', 1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 27: '<', 28: '>'}


In [0]:
trn_ds.vocab_size

29

In [0]:
x, y = trn_ds[1]

In [0]:
x.shape

torch.Size([9, 29])

In [0]:
y.shape

torch.Size([9])

Опишем модель, функцию потерь и алгоритм оптимизации

In [0]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.dropout = nn.Dropout(0.3)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
    
    def forward(self, h_prev, x):
        combined = torch.cat([h_prev, x], dim = 1) # concatenate x and h
        h = torch.tanh(self.dropout(self.i2h(combined)))
        y = self.i2o(h) #combined
        return h, y

In [0]:
model = RNN(trn_ds.vocab_size, hidden_size, trn_ds.vocab_size).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=1e-2)

![rnn](https://github.com/lizagonch/NLP/blob/master/4/images/dinos3.png?raw=1)

In [0]:
def sample(model):
    model.eval()
    word_size=0
    newline_idx = trn_ds.ch_to_idx['>']
    with torch.no_grad():
        h_prev = torch.zeros([1, hidden_size], dtype=torch.float, device=device)
        x = h_prev.new_zeros([1, trn_ds.vocab_size])
        start_char_idx = trn_ds.ch_to_idx['<']
        indices = [start_char_idx]
        x[0, start_char_idx] = 1
        predicted_char_idx = start_char_idx
        
        while predicted_char_idx != newline_idx and word_size != 50:
            h_prev, y_pred = model(h_prev, x)
            y_softmax_scores = torch.softmax(y_pred, dim=1)
            
            np.random.seed(np.random.randint(1, 5000))
            idx = np.random.choice(np.arange(trn_ds.vocab_size), p=y_softmax_scores.cpu().numpy().ravel())
            indices.append(idx)
            
            x = (y_pred == y_pred.max(1)[0]).float()
 
            
            predicted_char_idx = idx
            
            word_size += 1
        
        if word_size == 50:
            indices.append(newline_idx)
    return indices

In [0]:
def print_sample(sample_idxs):
    [print(trn_ds.idx_to_ch[x], end ='') for x in sample_idxs]
    print()

Обучим получившуюся модель

In [0]:
def train_one_epoch(model, loss_fn, optimizer):
    model.train()
    for line_num, (x, y) in enumerate(trn_dl):
        loss = 0
        optimizer.zero_grad()
        h_prev = torch.zeros([1, hidden_size], dtype=torch.float, device=device)
        x, y = x.to(device), y.to(device)
        for i in range(x.shape[1]):
            h_prev, y_pred = model(h_prev, x[:, i])
            loss += loss_fn(y_pred, y[:, i])
            
        if (line_num+1) % 100 == 0:
            print_sample(sample(model))
        loss.backward()
        optimizer.step()

In [0]:
def train(model, loss_fn, optimizer, dataset='dinos', epochs=1):
    for e in range(1, epochs+1):
        print('Epoch:{}'.format(e))
        train_one_epoch(model, loss_fn, optimizer)
        print()

In [0]:
train(model, loss_fn, optimizer, epochs = 10)

Epoch:1
<scohsaorus>
<cumiatirus>
<busuraurus>
<lsrucaurus>
<smsadiuous>
<gnaucaurus>
<suraskusus>
<suapoerhurus>
<burasaurus>
<aptotaurus>
<rinysdurup>
<lcctbor>
<ahonasaurus>
<anurtgiurus>
<rrwupauris>

Epoch:2
<alrasaurus>
<andoomasaurus>
<sutolaurus>
<pusapalrus>
<euabsacrus>
<hcrarzstor>
<speusaurus>
<tanranitrus>
<btgbpaurus>
<antotaurus>
<runysaurus>
<lcluchuris>
<larasipaudi>
<puirosaurus>
<srrusauris>

Epoch:3
<atctaheurus>
<gnauchurus>
<suggosaurus>
<tanianocaurus>
<mbrataurus>
<xssaspurus>
<kyrososaurus>
<auluaiosaurus>
<aubitnurus>
<llrun>
<srrusauras>
<ahltamonaaurus>
<sciupudhurus>
<rnvurguros>
<amsasaurus>

Epoch:4
<arrunaerus>
<oudpspurus>
<yuiptocaurus>
<slsactuaus>
<gnbucaurus>
<sipkosaurus>
<tbroaaunus>
<ahsoates>
<hcsapyurus>
<soeosiyiurussusuusaun>
<ahrocaurus>
<scluptetscurus>
<yuisouaculus>
<slsagusaus>
<gnbucaurus>

Epoch:5
<surhs>
<kysos>
<tbrnaaunus>
<agonasaurur>
<antotmotsaurus>
<sprutaraurus>
<euaasacrus>
<hctcouaurus>
<llotiuaurus>
<ipbisaurus>
<sraomouros

### Task 2.
Rewrite the sampling function so that pangrams (words that contain each character of the alphabet only ones)

### Task 3.
Rewrite the sampling function so that is it is possible to change the sampling temperature

# Reference

1. Sampling in  RNN: https://nlp.stanford.edu/blog/maximum-likelihood-decoding-with-rnns-the-good-the-bad-and-the-ugly/
2. Coursera course (main source): https://github.com/furkanu/deeplearning.ai-pytorch/tree/master/5-%20Sequence%20Models
3. Coursera course (main source): https://github.com/Kulbear/deep-learning-coursera/blob/master/Sequence%20Models/Dinosaurus%20Island%20--%20Character%20level%20language%20model%20final%20-%20v3.ipynb
4. LSTM: http://colah.github.io/posts/2015-08-Understanding-LSTMs/

## Named Entity Recognition


#### Постановка задачи «sequence labeling»:



* Дан корпус текстов $D$
* Каждый текст представляет собой последовательность токенов
* Каждому токену присвоена метка из некоторого множества $V$

В зависимости от множества меток $V$ получаем разные типы подзадач. Например:
* если $V$ - множество частей речи, то это задача ***POS***-теггинга
* если $V$ - множество типов именованных сущностей, то это задача ***NER***

Именованная сущность - любой фрагмент текста, обозначающий некоторый интересный объект.

#### Conditional Random Fields

***Conditional Random Field*** - развитие метода Марковских случайных полей. Графовая модель, которая используется для представления совместных распределений набора нескольких случайных переменных. 

Особенности модели ***CRF***:

* Качество сильно зависит от выбора признаков

* Один из лучших методов для ***NER*** и ***POS***-теггинга

* Долго обучается

* Хорошо работает в связке с рекуррентными нейросетями, моделирует совместное распределение на всей последовательности выходов сети одновременно

#### Архитектура BiLSTM-CRF

В данной модели для каждого слова вычисляется его векторное представление на основе символьного состава слова, предобученных векоторных представлений (***Word2Vec***, ***FastText***, ***GloVe***), а также других признаков (***POS***-тег, роль в предложении и т.д.)

![representation](https://github.com/lizagonch/NLP/blob/master/4/images/word_representation_model.png?raw=1)

Общая схема модели

![bilstm_crf](https://github.com/lizagonch/NLP/blob/master/4/images/bilstm_crf_model.png?raw=1)

Основные шаги алгоритма:
* Получить предобученные эмбеддинги слов коллекции (***word2vec***, ***GloVe***)
$$$$
* Обучить символьные эмбеддинги (***char-BiLSTM***, ***char-CNN***)
$$$$
* Составить для каждого слова морфологические/синтаксические признаки (***POS***-тег, роль в предложении и т.п.)
$$$$
* Объединить всё это и подать на вход основной сети (***BiLSTM***)
$$$$
* Выходы $h_t$ для всех слов предложения подавать на вход классификатору,
который будет предсказывать NER-тег (***SoftMax***, ***CRF***)

In [1]:
!pip install http://download.pytorch.org/whl/cu92/torch-0.4.1-cp36-cp36m-linux_x86_64.whl

     |████████████████████████████████| 512.6MB 1.1MB/s 
ERROR: torchvision 0.3.0 has requirement torch>=1.1.0, but you'll have torch 0.4.1 which is incompatible.
ERROR: fastai 1.0.57 has requirement torch>=1.0.0, but you'll have torch 0.4.1 which is incompatible.
  Found existing installation: torch 1.2.0
    Uninstalling torch-1.2.0:
      Successfully uninstalled torch-1.2.0


In [2]:
!pip install torchvision==0.2.0

     |████████████████████████████████| 51kB 2.0MB/s 
ERROR: fastai 1.0.57 has requirement torch>=1.0.0, but you'll have torch 0.4.1 which is incompatible.
  Found existing installation: torchvision 0.3.0
    Uninstalling torchvision-0.3.0:
      Successfully uninstalled torchvision-0.3.0


In [3]:
!pip install torchtext

In [4]:
!pip install natasha

In [5]:
!python -m spacy download en_core_web_sm

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [6]:
!pip install html5lib

In [7]:
!wget https://raw.githubusercontent.com/synalp/NER/master/corpus/CoNLL-2003/eng.testa

--2019-09-27 19:41:50--  https://raw.githubusercontent.com/synalp/NER/master/corpus/CoNLL-2003/eng.testa
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 827012 (808K) [text/plain]
Saving to: ‘eng.testa’

eng.testa           100%[===================>] 807.63K  --.-KB/s    in 0.07s   

2019-09-27 19:41:51 (10.6 MB/s) - ‘eng.testa’ saved [827012/827012]



In [8]:
!wget https://raw.githubusercontent.com/synalp/NER/master/corpus/CoNLL-2003/eng.testb

--2019-09-27 19:41:52--  https://raw.githubusercontent.com/synalp/NER/master/corpus/CoNLL-2003/eng.testb
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 748096 (731K) [text/plain]
Saving to: ‘eng.testb’

eng.testb           100%[===================>] 730.56K  --.-KB/s    in 0.06s   

2019-09-27 19:41:53 (11.4 MB/s) - ‘eng.testb’ saved [748096/748096]



In [9]:
!wget https://raw.githubusercontent.com/synalp/NER/master/corpus/CoNLL-2003/eng.train

--2019-09-27 19:41:54--  https://raw.githubusercontent.com/synalp/NER/master/corpus/CoNLL-2003/eng.train
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3281528 (3.1M) [text/plain]
Saving to: ‘eng.train’

eng.train           100%[===================>]   3.13M  --.-KB/s    in 0.1s    

2019-09-27 19:41:55 (28.2 MB/s) - ‘eng.train’ saved [3281528/3281528]



In [10]:
!mkdir datasets && mv eng.* datasets/

mkdir: cannot create directory ‘datasets’: File exists


In [11]:
!wget https://www.dropbox.com/s/rgfj9jzn24tgjkk/glove.6B.100d.txt
!mkdir embeddings 

--2019-09-27 19:41:59--  https://www.dropbox.com/s/rgfj9jzn24tgjkk/glove.6B.100d.txt
Resolving www.dropbox.com (www.dropbox.com)... 162.125.9.1, 2620:100:601f:1::a27d:901
Connecting to www.dropbox.com (www.dropbox.com)|162.125.9.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/rgfj9jzn24tgjkk/glove.6B.100d.txt [following]
--2019-09-27 19:41:59--  https://www.dropbox.com/s/raw/rgfj9jzn24tgjkk/glove.6B.100d.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc2869b6c9a9d5dd09c831f0bd88.dl.dropboxusercontent.com/cd/0/inline/ApXCsRXLQhdmNzkjR8rqzA8hvoLxRWvDX8JFWnciWnJQ_t9YOw1_GVhSUHYGdq6UMe-7if9-ojqo2rMuwg0Sqs_qjR3EiiR6z9HD_WqbE7fBpPabbmc-ni5H08ALkhWma3M/file# [following]
--2019-09-27 19:41:59--  https://uc2869b6c9a9d5dd09c831f0bd88.dl.dropboxusercontent.com/cd/0/inline/ApXCsRXLQhdmNzkjR8rqzA8hvoLxRWvDX8JFWnciWnJQ_t9YOw1_GVhSUHYGdq6UMe-7if9-ojqo2rMuwg0Sqs_qjR3EiiR6z9H

In [0]:
!mv glove.6B.100d.txt embeddings

In [13]:
!git clone https://github.com/sgrvinod/a-PyTorch-Tutorial-to-Sequence-Labeling

fatal: destination path 'a-PyTorch-Tutorial-to-Sequence-Labeling' already exists and is not an empty directory.


In [14]:
!mkdir conll2003 && cp datasets/eng.testa conll2003/eng.testa.txt && cp datasets/eng.testb conll2003/eng.testb.txt && cp datasets/eng.train conll2003/eng.train.txt

mkdir: cannot create directory ‘conll2003’: File exists


In [15]:
!mkdir .data && mv conll2003 .data/

mkdir: cannot create directory ‘.data’: File exists


In [16]:
!git clone https://github.com/kolloldas/torchnlp

fatal: destination path 'torchnlp' already exists and is not an empty directory.


In [17]:
!cd torchnlp && pip install -r requirements.txt && python setup.py install

  Cloning git://github.com/kolloldas/text.git to /tmp/pip-req-build-5ub1y0x7
  Running command git clone -q git://github.com/kolloldas/text.git /tmp/pip-req-build-5ub1y0x7
  Created wheel for torchtext: filename=torchtext-0.3.0-cp36-none-any.whl size=57517 sha256=de7732656fd892d84cdc5b9fb780ea20b05a0452e8cec129fcdb5ebf0e45864b
  Stored in directory: /tmp/pip-ephem-wheel-cache-e87y_o5w/wheels/e9/73/43/84b2351c8dc2cfb34334d033b497dd0161e9c3d4b6d3a0443d
Successfully built torchtext
running install
running bdist_egg
running egg_info
writing torchnlp.egg-info/PKG-INFO
writing dependency_links to torchnlp.egg-info/dependency_links.txt
writing top-level names to torchnlp.egg-info/top_level.txt
writing manifest file 'torchnlp.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/torchnlp
creating build/bdist.linux-x86_64/egg/torchnlp/common
copying build/lib/

#### Spacy 

In [0]:
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()

In [19]:
doc = nlp('European authorities fined Google a record $5.1 billion on Wednesday for abusing its power in the mobile phone market and ordered the company to alter its practices')
print([(X.text, X.label_) for X in doc.ents])

[('European', 'NORP'), ('Google', 'ORG'), ('$5.1 billion', 'MONEY'), ('Wednesday', 'DATE')]


Выкачаем статью и найдём в ней именованные сущности, выведем их число:

In [20]:
from bs4 import BeautifulSoup
import requests
import re
def url_to_string(url):
    res = requests.get(url)
    html = res.text
    soup = BeautifulSoup(html, 'html.parser')
    for script in soup(["script", "style", 'aside']):
        script.extract()
    return " ".join(re.split(r'[\n\t]+', soup.get_text()))
ny_bb = url_to_string('https://www.nytimes.com/2018/08/13/us/politics/peter-strzok-fired-fbi.html?hp&action=click&pgtype=Homepage&clickSource=story-heading&module=first-column-region&region=top-news&WT.nav=top-news')
article = nlp(ny_bb)
len(article.ents)

173

Выведем число встреченных сущностей каждого типа:

In [21]:
labels = [x.label_ for x in article.ents]
Counter(labels)

Counter({'CARDINAL': 5,
         'DATE': 23,
         'FAC': 1,
         'GPE': 16,
         'LAW': 1,
         'NORP': 2,
         'ORDINAL': 1,
         'ORG': 39,
         'PERSON': 82,
         'PRODUCT': 3})

Выведем текст с подсвеченными сущностями разных типов:

In [22]:
sentences = [x for x in article.sents]
displacy.render(nlp(str(sentences)), jupyter=True, style='ent')

In [0]:
import time
import torch
import torch.optim as optim
import os
import sys

sys.path.append('./a-PyTorch-Tutorial-to-Sequence-Labeling')

from models import LM_LSTM_CRF, ViterbiLoss
from utils import *
from torch.nn.utils.rnn import pack_padded_sequence
from datasets import WCDataset
from inference import ViterbiDecoder
from sklearn.metrics import f1_score

In [0]:
#!touch ./a-PyTorch-Tutorial-to-Sequence-Labeling/__init__.py

In [0]:
# Data parameters
task = 'ner'  # tagging task, to choose column in CoNLL 2003 dataset
train_file = './datasets/eng.train'  # path to training data
val_file = './datasets/eng.testa'  # path to validation data
test_file = './datasets/eng.testb'  # path to test data
emb_file = './embeddings/glove.6B.100d.txt'  # path to pre-trained word embeddings
min_word_freq = 5  # threshold for word frequency
min_char_freq = 1  # threshold for character frequency
caseless = True  # lowercase everything?
expand_vocab = True  # expand model's input vocabulary to the pre-trained embeddings' vocabulary?

# Model parameters
char_emb_dim = 30  # character embedding size
with open(emb_file, 'r') as f:
    word_emb_dim = len(f.readline().split(' ')) - 1  # word embedding size
word_rnn_dim = 300  # word RNN size
char_rnn_dim = 300  # character RNN size
char_rnn_layers = 1  # number of layers in character RNN
word_rnn_layers = 1  # number of layers in word RNN
highway_layers = 1  # number of layers in highway network
dropout = 0.5  # dropout
fine_tune_word_embeddings = False  # fine-tune pre-trained word embeddings?

# Training parameters
start_epoch = 0  # start at this epoch
batch_size = 10  # batch size
lr = 0.015  # learning rate
lr_decay = 0.05  # decay learning rate by this amount
momentum = 0.9  # momentum
workers = 1  # number of workers for loading data in the DataLoader
epochs = 10  # number of epochs to run without early-stopping
grad_clip = 5.  # clip gradients at this value
print_freq = 100  # print training or validation status every __ batches
best_f1 = 0.  # F1 score to start with
checkpoint = None  # path to model checkpoint, None if none

tag_ind = 1 if task == 'pos' else 3  # choose column in CoNLL 2003 dataset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [0]:
def train(train_loader, model, lm_criterion, crf_criterion, optimizer, epoch, vb_decoder):
    """
    Performs one epoch's training.
    :param train_loader: DataLoader for training data
    :param model: model
    :param lm_criterion: cross entropy loss layer
    :param crf_criterion: viterbi loss layer
    :param optimizer: optimizer
    :param epoch: epoch number
    :param vb_decoder: viterbi decoder (to decode and find F1 score)
    """

    model.train()  # training mode enables dropout

    batch_time = AverageMeter()  # forward prop. + back prop. time per batch
    data_time = AverageMeter()  # data loading time per batch
    ce_losses = AverageMeter()  # cross entropy loss
    vb_losses = AverageMeter()  # viterbi loss
    f1s = AverageMeter()  # f1 score

    start = time.time()

    # Batches
    for i, (wmaps, cmaps_f, cmaps_b, cmarkers_f, cmarkers_b, tmaps, wmap_lengths, cmap_lengths) in enumerate(
            train_loader):

        data_time.update(time.time() - start)

        max_word_len = max(wmap_lengths.tolist())
        max_char_len = max(cmap_lengths.tolist())

        # Reduce batch's padded length to maximum in-batch sequence
        # This saves some compute on nn.Linear layers (RNNs are unaffected, since they don't compute over the pads)
        wmaps = wmaps[:, :max_word_len].to(device)
        cmaps_f = cmaps_f[:, :max_char_len].to(device)
        cmaps_b = cmaps_b[:, :max_char_len].to(device)
        cmarkers_f = cmarkers_f[:, :max_word_len].to(device)
        cmarkers_b = cmarkers_b[:, :max_word_len].to(device)
        tmaps = tmaps[:, :max_word_len].to(device)
        wmap_lengths = wmap_lengths.to(device)
        cmap_lengths = cmap_lengths.to(device)

        # Forward prop.
        crf_scores, lm_f_scores, lm_b_scores, wmaps_sorted, tmaps_sorted, wmap_lengths_sorted, _, __ = model(cmaps_f,
                                                                                                             cmaps_b,
                                                                                                             cmarkers_f,
                                                                                                             cmarkers_b,
                                                                                                             wmaps,
                                                                                                             tmaps,
                                                                                                             wmap_lengths,
                                                                                                             cmap_lengths)

        # LM loss

        # We don't predict the next word at the pads or <end> tokens
        # We will only predict at [dunston, checks, in] among [dunston, checks, in, <end>, <pad>, <pad>, ...]
        # So, prediction lengths are word sequence lengths - 1
        lm_lengths = wmap_lengths_sorted - 1
        lm_lengths = lm_lengths.tolist()

        # Remove scores at timesteps we won't predict at
        # pack_padded_sequence is a good trick to do this (see dynamic_rnn.py, where we explore this)
        lm_f_scores, _ = pack_padded_sequence(lm_f_scores, lm_lengths, batch_first=True)
        lm_b_scores, _ = pack_padded_sequence(lm_b_scores, lm_lengths, batch_first=True)

        # For the forward sequence, targets are from the second word onwards, up to <end>
        # (timestep -> target) ...dunston -> checks, ...checks -> in, ...in -> <end>
        lm_f_targets = wmaps_sorted[:, 1:]
        lm_f_targets, _ = pack_padded_sequence(lm_f_targets, lm_lengths, batch_first=True)

        # For the backward sequence, targets are <end> followed by all words except the last word
        # ...notsnud -> <end>, ...skcehc -> dunston, ...ni -> checks
        lm_b_targets = torch.cat(
            [torch.LongTensor([word_map['<end>']] * wmaps_sorted.size(0)).unsqueeze(1).to(device), wmaps_sorted], dim=1)
        lm_b_targets, _ = pack_padded_sequence(lm_b_targets, lm_lengths, batch_first=True)

        # Calculate loss
        ce_loss = lm_criterion(lm_f_scores, lm_f_targets) + lm_criterion(lm_b_scores, lm_b_targets)
        vb_loss = crf_criterion(crf_scores, tmaps_sorted, wmap_lengths_sorted)
        loss = ce_loss + vb_loss

        # Back prop.
        optimizer.zero_grad()
        loss.backward()

        if grad_clip is not None:
            clip_gradient(optimizer, grad_clip)

        optimizer.step()

        # Viterbi decode to find accuracy / f1
        decoded = vb_decoder.decode(crf_scores.to("cpu"), wmap_lengths_sorted.to("cpu"))

        # Remove timesteps we won't predict at, and also <end> tags, because to predict them would be cheating
        decoded, _ = pack_padded_sequence(decoded, lm_lengths, batch_first=True)
        tmaps_sorted = tmaps_sorted % vb_decoder.tagset_size  # actual target indices (see create_input_tensors())
        tmaps_sorted, _ = pack_padded_sequence(tmaps_sorted, lm_lengths, batch_first=True)

        # F1
        f1 = f1_score(tmaps_sorted.to("cpu").numpy(), decoded.numpy(), average='macro')

        # Keep track of metrics
        ce_losses.update(ce_loss.item(), sum(lm_lengths))
        vb_losses.update(vb_loss.item(), crf_scores.size(0))
        batch_time.update(time.time() - start)
        f1s.update(f1, sum(lm_lengths))

        start = time.time()

        # Print training status
        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Batch Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data Load Time {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'CE Loss {ce_loss.val:.4f} ({ce_loss.avg:.4f})\t'
                  'VB Loss {vb_loss.val:.4f} ({vb_loss.avg:.4f})\t'
                  'F1 {f1.val:.3f} ({f1.avg:.3f})'.format(epoch, i, len(train_loader),
                                                          batch_time=batch_time,
                                                          data_time=data_time, ce_loss=ce_losses,
                                                          vb_loss=vb_losses, f1=f1s))

In [0]:
def validate(val_loader, model, crf_criterion, vb_decoder):
    """
    Performs one epoch's validation.
    :param val_loader: DataLoader for validation data
    :param model: model
    :param crf_criterion: viterbi loss layer
    :param vb_decoder: viterbi decoder
    :return: validation F1 score
    """
    model.eval()

    batch_time = AverageMeter()
    vb_losses = AverageMeter()
    f1s = AverageMeter()

    start = time.time()

    for i, (wmaps, cmaps_f, cmaps_b, cmarkers_f, cmarkers_b, tmaps, wmap_lengths, cmap_lengths) in enumerate(
            val_loader):

        max_word_len = max(wmap_lengths.tolist())
        max_char_len = max(cmap_lengths.tolist())

        # Reduce batch's padded length to maximum in-batch sequence
        # This saves some compute on nn.Linear layers (RNNs are unaffected, since they don't compute over the pads)
        wmaps = wmaps[:, :max_word_len].to(device)
        cmaps_f = cmaps_f[:, :max_char_len].to(device)
        cmaps_b = cmaps_b[:, :max_char_len].to(device)
        cmarkers_f = cmarkers_f[:, :max_word_len].to(device)
        cmarkers_b = cmarkers_b[:, :max_word_len].to(device)
        tmaps = tmaps[:, :max_word_len].to(device)
        wmap_lengths = wmap_lengths.to(device)
        cmap_lengths = cmap_lengths.to(device)

        # Forward prop.
        crf_scores, wmaps_sorted, tmaps_sorted, wmap_lengths_sorted, _, __ = model(cmaps_f,
                                                                                   cmaps_b,
                                                                                   cmarkers_f,
                                                                                   cmarkers_b,
                                                                                   wmaps,
                                                                                   tmaps,
                                                                                   wmap_lengths,
                                                                                   cmap_lengths)

        # Viterbi / CRF layer loss
        vb_loss = crf_criterion(crf_scores, tmaps_sorted, wmap_lengths_sorted)

        # Viterbi decode to find accuracy / f1
        decoded = vb_decoder.decode(crf_scores.to("cpu"), wmap_lengths_sorted.to("cpu"))

        # Remove timesteps we won't predict at, and also <end> tags, because to predict them would be cheating
        decoded, _ = pack_padded_sequence(decoded, (wmap_lengths_sorted - 1).tolist(), batch_first=True)
        tmaps_sorted = tmaps_sorted % vb_decoder.tagset_size  # actual target indices (see create_input_tensors())
        tmaps_sorted, _ = pack_padded_sequence(tmaps_sorted, (wmap_lengths_sorted - 1).tolist(), batch_first=True)

        # f1
        f1 = f1_score(tmaps_sorted.to("cpu").numpy(), decoded.numpy(), average='macro')

        # Keep track of metrics
        vb_losses.update(vb_loss.item(), crf_scores.size(0))
        f1s.update(f1, sum((wmap_lengths_sorted - 1).tolist()))
        batch_time.update(time.time() - start)

        start = time.time()

        if i % print_freq == 0:
            print('Validation: [{0}/{1}]\t'
                  'Batch Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'VB Loss {vb_loss.val:.4f} ({vb_loss.avg:.4f})\t'
                  'F1 Score {f1.val:.3f} ({f1.avg:.3f})\t'.format(i, len(val_loader), batch_time=batch_time,
                                                                  vb_loss=vb_losses, f1=f1s))

    print(
        '\n * LOSS - {vb_loss.avg:.3f}, F1 SCORE - {f1.avg:.3f}\n'.format(vb_loss=vb_losses,
                                                                          f1=f1s))

    return f1s.avg

In [0]:
def main_func():
    """
    Training and validation.
    """
    global best_f1, epochs_since_improvement, checkpoint, start_epoch, word_map, char_map, tag_map

    # Read training and validation data
    train_words, train_tags = read_words_tags(train_file, tag_ind, caseless)
    val_words, val_tags = read_words_tags(val_file, tag_ind, caseless)

    # Initialize model or load checkpoint
    if checkpoint is not None:
        checkpoint = torch.load(checkpoint)
        model = checkpoint['model']
        optimizer = checkpoint['optimizer']
        word_map = checkpoint['word_map']
        lm_vocab_size = checkpoint['lm_vocab_size']
        tag_map = checkpoint['tag_map']
        char_map = checkpoint['char_map']
        start_epoch = checkpoint['epoch'] + 1
        best_f1 = checkpoint['f1']
    else:
        word_map, char_map, tag_map = create_maps(train_words + val_words, train_tags + val_tags, min_word_freq,
                                                  min_char_freq)  # create word, char, tag maps
        embeddings, word_map, lm_vocab_size = load_embeddings(emb_file, word_map,
                                                              expand_vocab)  # load pre-trained embeddings

        model = LM_LSTM_CRF(tagset_size=len(tag_map),
                            charset_size=len(char_map),
                            char_emb_dim=char_emb_dim,
                            char_rnn_dim=char_rnn_dim,
                            char_rnn_layers=char_rnn_layers,
                            vocab_size=len(word_map),
                            lm_vocab_size=lm_vocab_size,
                            word_emb_dim=word_emb_dim,
                            word_rnn_dim=word_rnn_dim,
                            word_rnn_layers=word_rnn_layers,
                            dropout=dropout,
                            highway_layers=highway_layers).to(device)
        model.init_word_embeddings(embeddings.to(device))  # initialize embedding layer with pre-trained embeddings
        model.fine_tune_word_embeddings(fine_tune_word_embeddings)  # fine-tune
        optimizer = optim.SGD(params=filter(lambda p: p.requires_grad, model.parameters()), lr=lr, momentum=momentum)

    # Loss functions
    lm_criterion = nn.CrossEntropyLoss().to(device)
    crf_criterion = ViterbiLoss(tag_map).to(device)

    # Since the language model's vocab is restricted to in-corpus indices, encode training/val with only these!
    # word_map might have been expanded, and in-corpus words eliminated due to low frequency might still be added because
    # they were in the pre-trained embeddings
    temp_word_map = {k: v for k, v in word_map.items() if v <= word_map['<unk>']}
    train_inputs = create_input_tensors(train_words, train_tags, temp_word_map, char_map,
                                        tag_map)
    val_inputs = create_input_tensors(val_words, val_tags, temp_word_map, char_map, tag_map)

    # DataLoaders
    train_loader = torch.utils.data.DataLoader(WCDataset(*train_inputs), batch_size=batch_size, shuffle=True,
                                               num_workers=workers, pin_memory=False)
    val_loader = torch.utils.data.DataLoader(WCDataset(*val_inputs), batch_size=batch_size, shuffle=True,
                                             num_workers=workers, pin_memory=False)

    # Viterbi decoder (to find accuracy during validation)
    vb_decoder = ViterbiDecoder(tag_map)

    # Epochs
    for epoch in range(start_epoch, epochs):

        # One epoch's training
        train(train_loader=train_loader,
              model=model,
              lm_criterion=lm_criterion,
              crf_criterion=crf_criterion,
              optimizer=optimizer,
              epoch=epoch,
              vb_decoder=vb_decoder)

        # One epoch's validation
        val_f1 = validate(val_loader=val_loader,
                          model=model,
                          crf_criterion=crf_criterion,
                          vb_decoder=vb_decoder)

        # Did validation F1 score improve?
        is_best = val_f1 > best_f1
        best_f1 = max(val_f1, best_f1)
        if not is_best:
            epochs_since_improvement += 1
            print("\nEpochs since improvement: %d\n" % (epochs_since_improvement,))
        else:
            epochs_since_improvement = 0

        # Save checkpoint
        save_checkpoint(epoch, model, optimizer, val_f1, word_map, char_map, tag_map, lm_vocab_size, is_best)

        # Decay learning rate every epoch
        adjust_learning_rate(optimizer, lr / (1 + (epoch + 1) * lr_decay))

In [0]:
main_func()

Embedding length is 100.
You have elected to include embeddings that are out-of-corpus.

Loading embeddings...
'word_map' is being updated accordingly.

Done.
 Embedding vocabulary: 400054
 Language Model vocabulary: 4671.



/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:38: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1006: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Epoch: [0][0/1405]	Batch Time 1.371 (1.371)	Data Load Time 0.054 (0.054)	CE Loss 16.9024 (16.9024)	VB Loss 43.0178 (43.0178)	F1 0.023 (0.023)


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Epoch: [0][100/1405]	Batch Time 0.860 (1.081)	Data Load Time 0.005 (0.005)	CE Loss 13.6480 (15.9571)	VB Loss 12.1576 (8.2477)	F1 0.360 (0.323)
Epoch: [0][200/1405]	Batch Time 1.194 (1.063)	Data Load Time 0.005 (0.005)	CE Loss 12.7072 (14.6285)	VB Loss 4.6557 (6.7374)	F1 0.464 (0.402)
Epoch: [0][300/1405]	Batch Time 1.113 (1.061)	Data Load Time 0.005 (0.005)	CE Loss 12.7066 (13.9654)	VB Loss 4.5595 (5.9625)	F1 0.732 (0.454)
Epoch: [0][400/1405]	Batch Time 0.457 (1.058)	Data Load Time 0.004 (0.005)	CE Loss 11.2248 (13.5392)	VB Loss 1.8214 (5.4956)	F1 0.664 (0.493)
Epoch: [0][500/1405]	Batch Time 1.252 (1.058)	Data Load Time 0.005 (0.005)	CE Loss 12.3844 (13.2598)	VB Loss 5.3969 (5.2004)	F1 0.688 (0.519)
Epoch: [0][600/1405]	Batch Time 1.119 (1.058)	Data Load Time 0.004 (0.005)	CE Loss 12.5345 (13.0695)	VB Loss 3.7420 (4.9286)	F1 0.394 (0.538)
Epoch: [0][700/1405]	Batch Time 1.464 (1.059)	Data Load Time 0.005 (0.005)	CE Loss 12.1667 (12.9194)	VB Loss 3.9325 (4.7203)	F1 0.629 (0.551)
Epoch

In [42]:
torch.__version__
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130


In [43]:
!git clone https://gist.github.com/f7b7c7758a46da49f84bc68b47997d69.git

Cloning into 'f7b7c7758a46da49f84bc68b47997d69'...
remote: Enumerating objects: 16, done.
remote: Total 16 (delta 0), reused 0 (delta 0), pack-reused 16
Unpacking objects: 100% (16/16), done.


In [44]:
!bash pytorch041_cuda92_colab.sh

bash: pytorch041_cuda92_colab.sh: No such file or directory
